<a href="https://colab.research.google.com/github/Farwa-01/SEG_Training_Results_60_Site_DKA_M18_A_Phase/blob/main/AdvancedParameters_4_PC_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, explained_variance_score
import time

# Mount Google Drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/SEG Forecast/60-Site_DKA-M18_A-Phase.csv')

# Drop rows with missing values
data.dropna(inplace=True)

# Drop non-numeric columns
data_numeric = data.select_dtypes(include=[np.number])
data_numeric.dropna(inplace=True)

# Calculate Pearson correlation coefficients
correlation_matrix = data_numeric.corr(method='pearson')

# Splitting the dataset into input variables (X) and target variable (y)
X = data[['Global_Horizontal_Radiation', 'Diffuse_Horizontal_Radiation',
                  'Weather_Temperature_Celsius', 'Wind_Speed']]
y = data['Active_Power']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating an SVR model with specified parameters
svr_model = SVR(C=1, kernel='rbf', gamma='scale')

# Training the model and measuring the training time
start_time = time.time()
svr_model.fit(X_train, y_train)
end_time = time.time()
training_time = end_time - start_time

# Predicting on the testing set
y_pred = svr_model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

# Calculate Huber Loss
from sklearn.linear_model import HuberRegressor

huber_model = HuberRegressor()
huber_model.fit(X_train, y_train)
y_pred_huber = huber_model.predict(X_test)
def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = np.abs(error) <= delta
    squared_loss = np.square(error) / 2
    linear_loss = delta * (np.abs(error) - delta / 2)
    return np.where(is_small_error, squared_loss, linear_loss).mean()
huber_loss_value = huber_loss(y_test, y_pred_huber)

print("Support Vector Regression Model")
print("Training Time:", training_time)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Median Absolute Error:", medae)
print("Explained Variance Score:", evs)
print("Huber Loss:", huber_loss_value)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Support Vector Regression Model
Training Time: 1141.1516938209534
Mean Squared Error: 0.14506780246245254
R-squared: 0.9554980849884647
Median Absolute Error: 0.08236913873640717
Explained Variance Score: 0.9555612136629974
Huber Loss: 0.074776333502598


In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.17225976471051072


In [ ]:

from sklearn.metrics import mean_absolute_percentage_error


# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)


# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)


Root Mean Squared Error (RMSE): 0.380877673882905
Mean Absolute Percentage Error (MAPE): 174371477970328.7
